### Load Python Libraries

In [30]:
# Load libraries
import numpy
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
%matplotlib inline

### Load the Gastric Cancer Dataset

In [36]:
dataset_full = read_csv("./covariate_genome_freq_table_nonMSI_prefiltered.csv")

In [37]:
dataset = dataset_full.drop(['Unnamed: 0', 'sites.sid'], axis=1)
dataset.tail()

,local_mutrate,mean_rep_time,H3K4Me1_tumor,H3K36me3_E111,H3K4me3_E094,DNase_E094,H3K27me3_E094,H3K9me3_E111,H2BK15ac_meta,H3K4me2_meta,...,H3K23ac_meta,SMC3,threeMer,oneMer,threeRight,threeLeft,fiveRight,fiveLeft,mut.count,nonmut.count
30655595,0.000013,57.746571,0,0,0,0,1,0,0,0,...,0,0,AAC,A,0,AA,0,0,0,1
30655596,0.000013,57.746571,0,0,0,0,1,0,0,0,...,0,0,AAC,A,0,AA,0,0,0,1
30655597,0.000013,57.746571,0,0,0,0,1,0,0,0,...,0,0,AAC,A,0,AA,0,0,0,1
30655598,0.000013,57.746571,0,0,0,0,1,0,0,0,...,0,0,AAC,A,0,AA,0,0,0,1
30655599,0.000013,57.746571,0,0,0,0,1,0,0,0,...,0,0,AAC,A,0,AA,0,0,0,1


In [38]:
# shape
dataset.shape

(30655600, 23)

### Check if NaN exists

In [39]:
dataset.isnull().sum()

local_mutrate    0
mean_rep_time    0
H3K4Me1_tumor    0
H3K36me3_E111    0
H3K4me3_E094     0
DNase_E094       0
H3K27me3_E094    0
H3K9me3_E111     0
H2BK15ac_meta    0
H3K4me2_meta     0
H3K79me1_meta    0
H3T11ph_meta     0
CTCF             0
H3K23ac_meta     0
SMC3             0
threeMer         0
oneMer           0
threeRight       0
threeLeft        0
fiveRight        0
fiveLeft         0
mut.count        0
nonmut.count     0
dtype: int64

There is no NaN feed within the entire data. No preprocessing for NaN is required.

### Convert String values into Integer

In [40]:
# types
dataset.dtypes

local_mutrate    float64
mean_rep_time    float64
H3K4Me1_tumor      int64
H3K36me3_E111      int64
H3K4me3_E094       int64
DNase_E094         int64
H3K27me3_E094      int64
H3K9me3_E111       int64
H2BK15ac_meta      int64
H3K4me2_meta       int64
H3K79me1_meta      int64
H3T11ph_meta       int64
CTCF               int64
H3K23ac_meta       int64
SMC3               int64
threeMer          object
oneMer            object
threeRight        object
threeLeft         object
fiveRight         object
fiveLeft          object
mut.count          int64
nonmut.count       int64
dtype: object

In [41]:
seed = 7
numpy.random.seed(seed=seed)
rand_ix=numpy.random.randint(1, dataset.shape[0], size=10000)
rand_ix

array([25751728,  3335365, 27836954, ..., 11987222, 27683453,  2370571])

In [42]:
dataset_partial = dataset.iloc[rand_ix]
dataset_partial.tail()

,local_mutrate,mean_rep_time,H3K4Me1_tumor,H3K36me3_E111,H3K4me3_E094,DNase_E094,H3K27me3_E094,H3K9me3_E111,H2BK15ac_meta,H3K4me2_meta,...,H3K23ac_meta,SMC3,threeMer,oneMer,threeRight,threeLeft,fiveRight,fiveLeft,mut.count,nonmut.count
7390579,1.789195e-06,73.466195,1,0,1,0,0,0,0,1,...,0,0,AAG,A,0,AA,0,TAA,0,7
10909413,2.195768e-06,73.466195,0,0,1,0,0,0,0,0,...,0,0,0,0,0,CG,0,0,0,1561
11987222,2.572523e-06,23.354280,1,0,0,0,0,0,0,0,...,0,0,0,A,0,0,AGA,0,0,2
27683453,7.259739e-06,41.701398,0,0,0,0,0,0,0,0,...,0,1,0,A,0,AA,0,TAA,0,4
2370571,2.975448e-07,73.466195,0,0,0,0,0,0,0,0,...,0,1,0,A,0,AA,AAG,TAA,0,25


In [43]:
dataset = dataset_partial

# List up Multiclass string values and its count
obj_cols = ['threeMer', 'oneMer', 'threeRight', 'threeLeft', 'fiveRight', 'fiveLeft']

for i, v in enumerate(obj_cols):
    print(dataset.groupby(v).size())
    print 

threeMer
0      7792
AAC     656
AAG    1552
dtype: int64
oneMer
0    3282
A    6718
dtype: int64
threeRight
0     8539
GA    1461
dtype: int64
threeLeft
0     4189
AA    3472
CA    1572
CG     767
dtype: int64
fiveRight
0      6394
AAG    1259
AGA    1208
AGT    1139
dtype: int64
fiveLeft
0      7294
AAG     685
TAA    1279
TTG     742
dtype: int64


### Try MultiLabelBinarizer()...

In [48]:
dataset_enc = dataset
mlb = []
obj_cols = ['threeMer', 'oneMer', 'threeRight', 'threeLeft', 'fiveRight', 'fiveLeft']
for i, v in enumerate(obj_cols):    
    mlb.append(MultiLabelBinarizer())
    mlb[i].fit(dataset_enc[v])
    #dataset_enc[v] = mlb[i].transform(dataset[v])
    print("Number of classes for " + v + " : " + str(len(mlb[i].classes_)))
    #print mlb[i].classes_
dataset_enc.head()

TypeError: 'numpy.int64' object is not iterable

### Convert Categorical Variables into Integer first...

In [10]:
dataset_enc = dataset
mlb = []
obj_cols = ['threeMer', 'oneMer', 'threeRight', 'threeLeft', 'fiveRight', 'fiveLeft']
for i, v in enumerate(obj_cols):    
    mlb.append(LabelEncoder())
    mlb[i].fit(dataset[v])
    dataset_enc[v] = mlb[i].transform(dataset[v])
    print("Number of classes for " + v + " : " + str(len(mlb[i].classes_)))
    #print mlb[i].classes_
dataset_enc.head()

Number of classes for threeMer : 3
Number of classes for oneMer : 2
Number of classes for threeRight : 2
Number of classes for threeLeft : 4
Number of classes for fiveRight : 4
Number of classes for fiveLeft : 4


,local_mutrate,mean_rep_time,H3K4Me1_tumor,H3K36me3_E111,H3K4me3_E094,DNase_E094,H3K27me3_E094,H3K9me3_E111,H2BK15ac_meta,H3K4me2_meta,...,H3K23ac_meta,SMC3,threeMer,oneMer,threeRight,threeLeft,fiveRight,fiveLeft,mut.count,nonmut.count
25751728,0.000005,49.967968,0,0,1,0,0,0,0,1,...,0,0,0,1,0,1,1,0,0,18
3335365,0.000002,23.354280,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,310
27836954,0.000007,41.701398,0,0,1,0,0,0,0,1,...,0,0,2,1,0,1,2,0,0,184
27798007,0.000007,41.701398,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,1,0,0,4
3905092,0.000002,41.701398,1,0,0,0,0,0,0,0,...,0,0,2,1,0,1,0,0,0,1


### Then Apply One-Hot-Encoder

In [27]:
dataset['threeMer']

25751728    0
3335365     0
27836954    2
27798007    0
3905092     2
7131348     0
20251544    0
10565224    0
6128989     0
14281914    0
26020376    0
4812361     0
3461466     1
7467375     0
22805035    2
12085979    0
1225609     0
18585575    0
6931889     0
29480832    0
11657608    0
25516973    0
9952769     0
14758988    0
10545976    0
29616379    0
1899527     0
14360340    2
28052925    0
10513453    0
           ..
25279296    0
16802536    0
9870755     0
28623738    2
20615277    0
10955121    0
20129186    0
15861327    0
11980319    2
23654608    1
8773545     0
16007733    2
10886051    0
10368299    0
11580484    0
5787287     0
20546388    0
12987680    0
25854051    0
8808021     0
29456393    0
26517156    0
7070137     0
12885974    0
11223072    2
7390579     2
10909413    0
11987222    0
27683453    0
2370571     0
Name: threeMer, dtype: int64

In [26]:
mlb = OneHotEncoder()
hot = mlb.fit_transform(dataset['threeMer'])
print(hot)

  (0, 9999)	1.0
  (0, 9998)	1.0
  (0, 9997)	1.0
  (0, 9996)	1.0
  (0, 9995)	1.0
  (0, 9994)	1.0
  (0, 9993)	1.0
  (0, 9992)	1.0
  (0, 9991)	1.0
  (0, 9990)	1.0
  (0, 9989)	1.0
  (0, 9988)	1.0
  (0, 9987)	1.0
  (0, 9986)	1.0
  (0, 9985)	1.0
  (0, 9984)	1.0
  (0, 9983)	1.0
  (0, 9982)	1.0
  (0, 9981)	1.0
  (0, 9980)	1.0
  (0, 9979)	1.0
  (0, 9978)	1.0
  (0, 9977)	1.0
  (0, 9976)	1.0
  (0, 9975)	1.0
  :	:
  (0, 24)	1.0
  (0, 23)	1.0
  (0, 22)	1.0
  (0, 21)	1.0
  (0, 20)	1.0
  (0, 19)	1.0
  (0, 18)	1.0
  (0, 17)	1.0
  (0, 16)	1.0
  (0, 15)	1.0
  (0, 14)	1.0
  (0, 13)	1.0
  (0, 12)	1.0
  (0, 11)	1.0
  (0, 10)	1.0
  (0, 9)	1.0
  (0, 8)	1.0
  (0, 7)	1.0
  (0, 6)	1.0
  (0, 5)	1.0
  (0, 4)	1.0
  (0, 3)	1.0
  (0, 2)	1.0
  (0, 1)	1.0
  (0, 0)	1.0


/home/suns1/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [21]:
mlb = []
hot_enc = []
obj_cols = ['threeMer', 'oneMer', 'threeRight', 'threeLeft', 'fiveRight', 'fiveLeft']
for i, v in enumerate(obj_cols):    
    mlb.append(OneHotEncoder())
    hot_enc.append(mlb[i].fit_transform(dataset[v]))
    hot_enc[i]
    #dataset_enc[v] = mlb[i].transform(dataset[v])
#dataset_enc.head()

/home/suns1/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/suns1/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/suns1/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single samp

In [ ]:
#import pickle

#with open('dataset_enc.pickle', 'wb') as f:
#    pickle.dump(dataset_enc, f)

### Prepare Training and Validation Data

In [ ]:
import pickle
with open('dataset_enc.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
dataset.tail()

In [ ]:
seed = 7
numpy.random.seed(seed=seed)
rand_ix=numpy.random.randint(1, dataset.shape[0], size=10000)
rand_ix

In [ ]:
# Split-out validation dataset
dataset = dataset.values
input_dim = dataset.shape[1] - 1

# Use Entire Data
#X = dataset[:, 0:input_dim]
#Y = dataset[:, input_dim]

# Only Partial Data
X = dataset[rand_ix, 0:input_dim]
Y = dataset[rand_ix, input_dim]

validation_size = 0.20
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=7)

In [ ]:
# Base model for Feed Forward Neural Network using Keras Regressor
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
# Evaluate Algorithms
# Test options and evaluation metric
num_folds = 5
seed = 7
scoring = 'neg_mean_squared_error'

# Spot Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('RIDGE', Ridge()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVR', SVR()))
models.append(('XGB', XGBRegressor()))
models.append(('FFNN', KerasRegressor(build_fn=baseline_model, epochs=2000, batch_size=1024, verbose=0)))

# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure(figsize=(10, 7))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

#### Using Scaled Dataset

In [ ]:
# Test options and evaluation metric
num_folds = 5
seed = 7
scoring = 'neg_mean_squared_error'

# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
pipelines.append(('ScaledRIDGE', Pipeline([('Scaler', StandardScaler()),('RIDGE', Ridge())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB', XGBRegressor())])))
pipelines.append(('ScaledFFNN', Pipeline([('Scaler', StandardScaler()),('FFNN', KerasRegressor(build_fn=baseline_model, epochs=2000, batch_size=1024, verbose=0))])))

results = []
names = []

for name, model in pipelines:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure(figsize=(10, 7))
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
def plot_learning_curve(history):    
    print history.history
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.title('model loss - Mean Squared Error(MSE)')
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='upper right')
    pyplot.show()

# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

def deep_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

def deeper_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(64, kernel_initializer='normal', activation='relu'))
	model.add(Dense(32, kernel_initializer='normal', activation='relu'))
	model.add(Dense(8, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

### Prepare Neural Network Models

In [ ]:
def plot_learning_curve(history):    
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.title('model loss - Mean Squared Error(MSE)')
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='upper right')
    pyplot.show()

# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

def more_larger_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(64, kernel_initializer='normal', activation='relu'))
	model.add(Dense(32, kernel_initializer='normal', activation='relu'))
	model.add(Dense(8, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
# Evaluate Algorithms
# Test options and evaluation metric
num_folds = 3
seed = 7
scoring = 'neg_mean_squared_error'

# Spot Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('RIDGE', Ridge()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
#models.append(('SVR', SVR()))
models.append(('XGB', XGBRegressor()))
models.append(('FFNN', KerasRegressor(build_fn=baseline_model, epochs=2000, batch_size=1024, verbose=0)))

# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure(figsize=(10, 7))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
# Test options and evaluation metric
num_folds = 3
seed = 7
scoring = 'neg_mean_squared_error'

# Standardize the dataset
pipelines = []
#pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
pipelines.append(('ScaledRIDGE', Pipeline([('Scaler', StandardScaler()),('RIDGE', Ridge())])))
#pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
#pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
#pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
#pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
#pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())])))
#pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB', XGBRegressor())])))
#pipelines.append(('ScaledFFNN', Pipeline([('Scaler', StandardScaler()),('FFNN', KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=256))])))

results = []
names = []

for name, model in pipelines:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure(figsize=(10, 7))
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()